In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
from fashion_clip.fashion_clip import FashionCLIP
from typing import Literal

BATCH_SIZE = 32

fclip = FashionCLIP('fashion-clip')

In [2]:
long_attr_descr = {
    'cane_height_type': "Height of a cane, defining its length.",
    'closure_placement': "Position of garment closures like zippers or buttons.",
    'heel_shape_type': "Shape of a shoe's heel, e.g., block or stiletto.",
    'knit_structure': "Fabric knitting style, like ribbed or plain.",
    'length_type': "Length of a garment or part, e.g., cropped or full-length.",
    'neck_lapel_type': "Shape of necklines or lapels, e.g., V-neck or shawl.",
    'silhouette_type': "Overall garment shape, e.g., A-line or sheath.",
    'sleeve_length_type': "Sleeve length, e.g., short, long, or sleeveless.",
    'toecap_type': "Toe design in footwear, e.g., closed or open-toe.",
    'waist_type': "Design at the waist, e.g., high-waisted or elastic.",
    'woven_structure': "Weaving style, like twill or satin."
}

short_attr_descr = {
    'cane_height_type': "Cane height",
    'closure_placement': "Closure position",
    'heel_shape_type': "Heel shape",
    'knit_structure': "Knit pattern",
    'length_type': "Garment length",
    'neck_lapel_type': "Neckline style",
    'silhouette_type': "Garment shape",
    'sleeve_length_type': "Sleeve length",
    'toecap_type': "Toe design",
    'waist_type': "Waist style",
    'woven_structure': "Weave pattern"
}

long_subattr_descr = {
    'cane_height_type': {
        'Bloque': "Block-style heel, providing stability and comfort.",
        'Cuña': "Wedge heel, distributing weight evenly.",
        'Cuña abotinada': "Boot-style wedge heel for enhanced support.",
        'Alta': "High heel, adding height and elegance.",
        'Baja': "Low heel, prioritizing comfort and practicality.",
        'Media': "Mid-height heel, balancing style and comfort."
    },
    'closure_placement': {
        'Cuello': "Closure at the neck area, often with buttons or zippers.",
        'Sin cierre': "No closure, typically for pull-on garments.",
        'Cierre Delantero': "Front closure, usually with zippers or buttons.",
        'Cierre Trasero': "Back closure for discreet fastening.",
        'Cierre Hombro': "Shoulder closure, often seen in specific designs.",
        'Lateral': "Side closure, common for skirts or pants."
    },
    'heel_shape_type': {
        'Plano': "Flat heel, prioritizing comfort.",
        'Bloque': "Block heel, stable and supportive.",
        'Plataforma': "Platform heel, adding height with even support.",
        'Plataforma plana': "Flat platform, combining height with comfort.",
        'De aguja': "Stiletto heel, thin and elegant.",
        'Trompeta': "Trumpet-shaped heel, flaring at the bottom.",
        'Rectangular': "Rectangular heel, unique and bold.",
        'Kitten': "Kitten heel, low and dainty.",
        'Embudo': "Cone-shaped heel, tapering towards the bottom.",
        'Cuña': "Wedge heel, offering even support.",
        'Plataforma en la parte delantera': "Front platform, reducing the incline of high heels."
    },
    'knit_structure': {
        'Punto fino': "Fine knit, lightweight and delicate.",
        'Punto medio': "Medium knit, versatile and balanced.",
        'Punto grueso': "Thick knit, warm and cozy.",
        'Hecho a mano': "Hand-knit, often unique and artisanal."
    },
    'length_type': {
        'Largo': "Long length, extending fully.",
        'Corto': "Short length, above the knee or shorter.",
        'Standard': "Standard length, typically mid-thigh to knee.",
        'Crop': "Cropped length, stopping above the waist.",
        'Medio': "Medium length, between short and long.",
        'Midi': "Midi length, reaching mid-calf.",
        'Capri': "Capri length, ending just below the knee.",
        'Mini/Micro': "Mini or micro length, very short.",
        'Asimétrico': "Asymmetric length, varying hems.",
        'Maxi': "Maxi length, reaching the ankles or floor.",
        'Tres Cuartos': "Three-quarter length, stopping mid-shin.",
        'Tobillero': "Ankle-length, stopping at the ankles."
    },
    'neck_lapel_type': {
        'Hawaiano/Bowling': "Hawaiian or bowling-style collar, relaxed and open.",
        'Capucha': "Hooded neckline, functional and casual.",
        'Regular': "Standard neckline, versatile and common.",
        'Panadero': "Baker's collar, simple and practical.",
        'Cutaway': "Cutaway collar, wide and angled.",
        'Caja': "Box collar, square and structured.",
        'Pico': "Pointed collar, sharp and formal.",
        'Mao': "Mandarin collar, short and upright.",
        'Smoking': "Tuxedo collar, formal and elegant.",
        'Peak Lapel': "Pointed lapel, classic and tailored.",
        'Alto/Envolvente': "High or wrap-around collar, cozy and warm.",
        'Perkins': "Perkins collar, upright and fitted.",
        'Button Down': "Button-down collar, practical and neat.",
        'Halter': "Halter neckline, tied or fastened around the neck.",
        'Escotado': "Low-cut neckline, revealing.",
        'Redondo': "Round neckline, simple and classic.",
        'Polo': "Polo collar, sporty and casual.",
        'Camisero': "Shirt-style collar, structured and versatile.",
        'Chimenea': "Chimney collar, high and narrow.",
        'Cisne': "Swan neckline, high and elegant.",
        'Off Shoulder': "Off-shoulder neckline, exposing shoulders.",
        'Solapa': "Lapel, part of the jacket collar.",
        'Cruzado': "Crossed lapel, overlapping design.",
        'Shawl': "Shawl lapel, rounded and formal.",
        'Palabra Honor': "Strapless neckline, exposing shoulders and arms.",
        'Babydoll/Peter Pan': "Peter Pan collar, round and youthful.",
        'Drapeado': "Draped neckline, flowing fabric.",
        'Barca': "Boat neckline, wide and shallow.",
        'Waterfall': "Waterfall neckline, cascading folds.",
        'Asimétrico': "Asymmetric neckline, uneven design.",
        'Espalda Abierta': "Open back neckline, revealing the back.",
        'Kimono': "Kimono neckline, simple and wide.",
        'Sin solapa': "No lapel, minimalistic."
    },
    'silhouette_type': {
        'Regular': "Standard fit, neither tight nor loose.",
        'Slim': "Slim fit, close to the body.",
        '5 Bolsillos': "Five-pocket style, classic denim design.",
        'Jogger': "Jogger style, sporty and casual.",
        'Modern slim': "Modern slim fit, tailored yet comfortable.",
        'Chino': "Chino style, versatile and semi-formal.",
        'Recto': "Straight fit, consistent width throughout.",
        'Slouchy': "Relaxed and oversized fit, comfortable.",
        'Skinny': "Tight fit, closely contouring the body.",
        'Acampanado/Flare': "Flared style, widening from the knees.",
        'Push Up': "Tight fit, enhancing curves.",
        'Mom': "High-waisted, relaxed fit style.",
        'Evase': "A-line style, gently flaring out.",
        'Culotte': "Wide-legged, cropped trousers.",
        'Palazzo': "Very wide-legged pants, flowing and elegant.",
        'Acampanado/Bootcut': "Bootcut style, slight flare at the hem.",
        'Cargo': "Cargo style, with multiple utility pockets.",
        'Boyfriend': "Relaxed fit, slightly oversized.",
        'Fino': "Slim and fine fit, closely tailored.",
        'Sarouel': "Harem-style pants, loose and draped.",
        'Lápiz': "Pencil fit, tight and narrow.",
        'Ancho': "Wide fit, loose and flowing.",
        'Oversize': "Extra-large fit, bold and comfortable.",
        'Halter': "Halter silhouette, tied or fastened around the neck.",
        'Wide leg': "Wide-leg trousers, flowing and dramatic.",
        'Paperbag': "High-waisted with a gathered or belted waist.",
        'Relaxed': "Casual fit, loose but not oversized.",
        'Tapered': "Tapered fit, narrowing towards the ankle.",
        'Bandeau': "Strapless and fitted, around the bust.",
        'Superslim': "Extremely slim fit, tight throughout.",
        'Loose': "Relaxed fit, prioritizing comfort.",
        'Carrot': "Carrot fit, loose at the top and tapered at the ankle.",
        'Parachute': "Very loose and baggy, inspired by parachute fabric."
    },
    'sleeve_length_type': {
        'Corta': "Short sleeves, ending above the elbow.",
        'Larga': "Long sleeves, covering the arms completely.",
        'Tirante Ancho': "Wide straps, covering the shoulders partially.",
        'Tirante Fino': "Thin straps, minimal coverage.",
        'Sin Manga': "Sleeveless, no coverage on the arms.",
        'Tres Cuartos': "Three-quarter sleeves, ending below the elbow."
    },
    'toecap_type': {
        'Redonda': "Round toecap, classic and versatile.",
        'Con punta': "Pointed toecap, elegant and sharp.",
        'Abierta': "Open toecap, exposing the toes.",
        'Cuadrada': "Square toecap, modern and unique."
    },
    'waist_type': {
        'Ajustable/Goma': "Elastic or adjustable waist, flexible fit.",
        'Regular Waist': "Standard waist, sitting naturally on the hips.",
        'High Waist': "High-rise waist, sitting above the hips.",
        'Low Waist': "Low-rise waist, sitting below the hips."
    },
    'woven_structure': {
        'Pesado': "Heavy weave, durable and thick.",
        'Ligero': "Light weave, airy and breathable.",
        'Medio': "Medium weave, balanced and versatile.",
        'Elástico': "Stretch weave, flexible and form-fitting."
    }
}

short_subattr_descr = {
    'cane_height_type': {
        'Bloque': "Block heel",
        'Cuña': "Wedge heel",
        'Cuña abotinada': "Boot wedge",
        'Alta': "High heel",
        'Baja': "Low heel",
        'Media': "Mid heel"
    },
    'closure_placement': {
        'Cuello': "Neck closure",
        'Sin cierre': "No closure",
        'Cierre Delantero': "Front closure",
        'Cierre Trasero': "Back closure",
        'Cierre Hombro': "Shoulder closure",
        'Lateral': "Side closure"
    },
    'heel_shape_type': {
        'Plano': "Flat heel",
        'Bloque': "Block heel",
        'Plataforma': "Platform heel",
        'Plataforma plana': "Flat platform",
        'De aguja': "Stiletto heel",
        'Trompeta': "Trumpet heel",
        'Rectangular': "Rectangular heel",
        'Kitten': "Kitten heel",
        'Embudo': "Cone heel",
        'Cuña': "Wedge heel",
        'Plataforma en la parte delantera': "Front platform"
    },
    'knit_structure': {
        'Punto fino': "Fine knit",
        'Punto medio': "Medium knit",
        'Punto grueso': "Thick knit",
        'Hecho a mano': "Hand-knit"
    },
    'length_type': {
        'Largo': "Long length",
        'Corto': "Short length",
        'Standard': "Standard length",
        'Crop': "Cropped length",
        'Medio': "Medium length",
        'Midi': "Midi length",
        'Capri': "Capri length",
        'Mini/Micro': "Mini length",
        'Asimétrico': "Asymmetric length",
        'Maxi': "Maxi length",
        'Tres Cuartos': "Three-quarter",
        'Tobillero': "Ankle length"
    },
    'neck_lapel_type': {
        'Hawaiano/Bowling': "Hawaiian collar",
        'Capucha': "Hooded collar",
        'Regular': "Standard collar",
        'Panadero': "Baker's collar",
        'Cutaway': "Cutaway collar",
        'Caja': "Box collar",
        'Pico': "Pointed collar",
        'Mao': "Mandarin collar",
        'Smoking': "Tuxedo collar",
        'Peak Lapel': "Peak lapel",
        'Alto/Envolvente': "Wrap collar",
        'Perkins': "Perkins collar",
        'Button Down': "Button-down",
        'Halter': "Halter neck",
        'Escotado': "Low-cut",
        'Redondo': "Round neck",
        'Polo': "Polo collar",
        'Camisero': "Shirt collar",
        'Chimenea': "Chimney neck",
        'Cisne': "Swan neck",
        'Off Shoulder': "Off-shoulder",
        'Solapa': "Lapel",
        'Cruzado': "Crossed lapel",
        'Shawl': "Shawl lapel",
        'Palabra Honor': "Strapless",
        'Babydoll/Peter Pan': "Peter Pan",
        'Drapeado': "Draped neck",
        'Barca': "Boat neck",
        'Waterfall': "Waterfall neck",
        'Asimétrico': "Asymmetric neck",
        'Espalda Abierta': "Open back",
        'Kimono': "Kimono neck",
        'Sin solapa': "No lapel"
    },
    'silhouette_type': {
        'Regular': "Standard fit",
        'Slim': "Slim fit",
        '5 Bolsillos': "Five pockets",
        'Jogger': "Jogger fit",
        'Modern slim': "Modern slim",
        'Chino': "Chino style",
        'Recto': "Straight fit",
        'Slouchy': "Loose fit",
        'Skinny': "Tight fit",
        'Acampanado/Flare': "Flared fit",
        'Push Up': "Lifted fit",
        'Mom': "Mom jeans",
        'Evase': "A-line fit",
        'Culotte': "Wide cropped",
        'Palazzo': "Flowy wide",
        'Acampanado/Bootcut': "Bootcut fit",
        'Cargo': "Cargo style",
        'Boyfriend': "Relaxed fit",
        'Fino': "Thin fit",
        'Sarouel': "Harem pants",
        'Lápiz': "Pencil shape",
        'Ancho': "Wide fit",
        'Oversize': "Oversized fit",
        'Halter': "Halter fit",
        'Wide leg': "Wide leg",
        'Paperbag': "Cinched waist",
        'Relaxed': "Relaxed fit",
        'Tapered': "Tapered fit",
        'Bandeau': "Strapless fit",
        'Superslim': "Extra slim",
        'Loose': "Loose fit",
        'Carrot': "Carrot shape",
        'Parachute': "Puffy fit"
    },
    'sleeve_length_type': {
        'Corta': "Short sleeve",
        'Larga': "Long sleeve",
        'Tirante Ancho': "Wide strap",
        'Tirante Fino': "Thin strap",
        'Sin Manga': "Sleeveless",
        'Tres Cuartos': "Three-quarter"
    },
    'toecap_type': {
        'Redonda': "Round toe",
        'Con punta': "Pointed toe",
        'Abierta': "Open toe",
        'Cuadrada': "Square toe"
    },
    'waist_type': {
        'Ajustable/Goma': "Elastic waist",
        'Regular Waist': "Standard waist",
        'High Waist': "High waist",
        'Low Waist': "Low waist"
    },
    'woven_structure': {
        'Pesado': "Heavy weave",
        'Ligero': "Light weave",
        'Medio': "Medium weave",
        'Elástico': "Stretch weave"
    }
}


In [3]:
def generate_embeddings(descr_dict: dict[str, str]) -> dict[str, np.ndarray]:
    key, descr = list(descr_dict.keys()), list(descr_dict.values())

    descr_embeddings = fclip.encode_text(descr, batch_size=BATCH_SIZE)
    
    return {k: v for k, v in zip(key, descr_embeddings)}

long_attr_embeddings = generate_embeddings(long_attr_descr)
short_attr_embeddings = generate_embeddings(short_attr_descr)

long_subattr_embeddings = {k: generate_embeddings(v) for k, v in long_subattr_descr.items()}
short_subattr_embeddings = {k: generate_embeddings(v) for k, v in short_subattr_descr.items()}

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

1it [00:00,  2.47it/s]


Map:   0%|          | 0/11 [00:00<?, ? examples/s]

1it [00:00,  2.40it/s]


Map:   0%|          | 0/6 [00:00<?, ? examples/s]

...

1it [00:00,  5.15it/s]


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

1it [00:00,  5.39it/s]


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

1it [00:00,  5.75it/s]


In [4]:
# save in text_embeddings/ folder

with open('text_embeddings/long_attr_embeddings.pkl', 'wb') as f:
    pkl.dump(long_attr_embeddings, f)

with open('text_embeddings/short_attr_embeddings.pkl', 'wb') as f:
    pkl.dump(short_attr_embeddings, f)

with open('text_embeddings/long_subattr_embeddings.pkl', 'wb') as f:
    pkl.dump(long_subattr_embeddings, f)

with open('text_embeddings/short_subattr_embeddings.pkl', 'wb') as f:
    pkl.dump(short_subattr_embeddings, f)